In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
from typing import List, Literal, Optional, Dict, Union, Tuple, TypedDict

import pandas as pd
from pydantic import BaseModel, Field
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain_community.document_loaders.csv_loader import CSVLoader
from openai import OpenAI
from openai.types import ImagesResponse
import requests
from lancedb.pydantic import Vector, LanceModel
import lancedb
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function
from langchain_openai import OpenAIEmbeddings
import dotenv

In [88]:
dotenv.load_dotenv()

True

In [89]:
openai_model_name = 'gpt-3.5-turbo'

## 1. Generate 20 Diverse Real Estate Listings

In [61]:
open_ai_llm = ChatOpenAI(model_name=openai_model_name, temperature=1.3, max_tokens=4000)

In [62]:
class RealEstateListingType(TypedDict):
    city: str
    neighborhood: str
    street_address: str
    state: str
    price: str
    school_rating: float
    bedrooms: int
    bathrooms: float
    house_size: str
    house_type: str
    lot_size: str
    has_garage: bool
    has_pool: bool
    has_garden: bool
    has_view: bool
    year_built: int
    special_features: str
    property_description: str
    neighborhood_description: str


class RealEstateListing(BaseModel):
    """Real Estate Listing"""
    city: str = Field(description="City where the property is located")
    neighborhood: str = Field(description="Name of the neighborhood")
    street_address: str = Field(
        description="A plausible street address of the property. start the address with a two digit number.")
    state: str = Field(description="State where the property is located")
    price: str = Field(description="A realistic listing price of the property, given the features and location")
    school_rating: float = Field(
        description="Average school rating of the neighborhood. Be realistic. Only really expensive neighborhoods can be rated 8 and more.")
    bedrooms: int = Field(description="Number of bedrooms (typically between 1 and 6).")
    bathrooms: float = Field(description="Number of bathrooms, including half baths (e.g., 2.5).")
    house_size: str = Field(description="Size of the house in square feet")
    house_type: Literal['single-family', 'condo', 'townhouse', 'apartment', 'multi-family'] = Field(
        description="Type of the house (e.g., single-family, condo, townhouse, apartment, multi-family)")
    lot_size: str = Field(description="Size of the lot in square feet")
    has_garage: bool = Field(description="Whether the property has a garage")
    has_pool: bool = Field(description="Whether the property has a pool")
    has_garden: bool = Field(description="Whether the property has a garden")
    has_view: bool = Field(description="Whether the property has a view")
    year_built: int = Field(description="Year the property was built")
    special_features: str = Field(description="Special features of the property")
    property_description: str = Field(
        description="A description of the property as if a real estate agent is describing it")
    neighborhood_description: str = Field(description="Description of the neighborhood that fits in one paragraph")


def create_template(model: BaseModel) -> ChatPromptTemplate:
    """Generates a ChatPromptTemplate based on a Pydantic model"""

    instructions = """Imagine you're a creative AI real estate assistant tasked with generating a captivating real estate listing from scratch. Your job is to create a realistic, attractive listing for a fictional property. Focus on making the description lively and appealing, similar to how a real estate agent might present it. Ensure all the fields below are included and filled with creative, plausible details that could entice potential buyers. Here is a general description of the property listing I want you to generate: {input}
    """

    properties = model.schema()["properties"]
    output_format = "\n".join(
        f"* {name} ({info['type']}): ({info['description']}): " for name, info in properties.items())

    return ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions),
        AIMessagePromptTemplate.from_template(output_format)
    ])


def convert_listings_to_df(results: List[RealEstateListingType]) -> None:
    """Converts a list of RealEstateListingType to a DataFrame"""
    df = pd.DataFrame(results).astype(RealEstateListingType.__annotations__)
    return df




In [63]:
prompt = create_template(RealEstateListing)
openai_real_estate_function = convert_pydantic_to_openai_function(RealEstateListing)
llm_model_with_functions = open_ai_llm.bind(functions=[openai_real_estate_function],
                                            function_call={"name": "RealEstateListing"})

In [64]:
tagging_chain = prompt | llm_model_with_functions | JsonOutputFunctionsParser()

In [65]:
batch_inputs = [
    {'input': 'I want the property to be a huge, expensive property in Cambridge, MA'},
    {'input': 'I want a cozy, small cottage in the woods of Vermont'},
    {'input': 'A tiny apartment in a challenging part of Roxbury, Boston, MA'},
    {'input': 'A nice house with a pool in La Jolla, California'},
    {'input': 'A modern loft apartment in downtown Austin, Texas'},
    {'input': 'A spacious villa with a private beach in Malibu, California'},
    {'input': 'An antique farmhouse with vast lands in Charlottesville, Virginia'},
    {'input': 'A luxurious penthouse overlooking Central Park in New York City, NY'},
    {'input': 'A secluded cabin in the Rocky Mountains of Colorado'},
    {'input': 'A historic mansion in the Garden District of New Orleans, Louisiana'},
    {'input': 'A sustainable eco-friendly home in Portland, Oregon'},
    {'input': 'A minimalist studio apartment in downtown Seattle, Washington'},
    {'input': 'A colonial-style house with a large porch in Charleston, South Carolina'},
    {'input': 'A classic townhouse in the Georgetown neighborhood of Washington, D.C.'},
    {'input': 'A beachfront bungalow in Key West, Florida'},
    {'input': 'A high-rise condo with city views in Chicago, Illinois'},
    {'input': 'A traditional adobe home in Santa Fe, New Mexico'},
    {'input': 'A quaint Victorian house in San Francisco, California'},
    {'input': 'A detached guest house in Beverly Hills, California, with access to movie star amenities'},
    {'input': 'A sprawling estate near the historical sites of Williamsburg, Virginia'}
]

In [66]:
results = tagging_chain.batch(batch_inputs)

In [67]:
listings_df = convert_listings_to_df(results)

In [68]:
listings_df

,city,neighborhood,street_address,state,price,school_rating,bedrooms,bathrooms,house_size,house_type,lot_size,has_garage,has_pool,has_garden,has_view,year_built,special_features,property_description,neighborhood_description
0,Cambridge,Harvard Square,35 Crimson Lane,MA,"$5,500,000",9.0,5,4.5,"6,000 sqft",single-family,0.75 acres,True,True,True,True,1998,"Smart home technology, home theater, sauna, ro...",Welcome to the epitome of luxury living in pre...,"Harvard Square is a vibrant, intellectual hub ..."
1,Exeter,Maple Woods,33 Forest View Lane,Vermont,"$225,000",7.0,2,1.5,"1,200 sqft",single-family,0.5 acres,True,False,True,True,1978,"Cozy fireplace, Tranquil woods setting","Escape to this charming, cozy cottage nestled ...",Located in the sought-after Maple Woods neighb...
2,Boston,Roxbury,28 Main Street,MA,"$250,000",3.0,1,1.0,650 sqft,apartment,N/A,False,False,False,False,1950,"Recently renovated, hardwood floors, city living",Welcome to this cozy and inviting 1-bedroom ap...,Roxbury is a vibrant neighborhood known for it...
3,La Jolla,Seaside Village,22 Oceanfront Drive,California,"$1,850,000",7.0,4,3.5,"3,200 sqft",single-family,"8,000 sqft",True,True,True,True,2005,"Oceanside living, stunning views, luxurious pool",Welcome to this exquisite oceanfront retreat l...,Seaside Village is known for its exclusive wat...
4,Austin,Downtown,12 Pleasant Street,Texas,"$650,000",7.0,2,2.5,"1,500 sqft",apartment,N/A,True,False,False,True,2015,"Modern design, rooftop terrace, city views",Welcome to this ultra-modern loft apartment lo...,Downtown Austin offers a vibrant urban lifesty...
5,Malibu,Sandy Shores,23 Ocean View Avenue,California,"$7,500,000",9.0,4,4.5,"4,500 sqft",single-family,0.5 acres,True,True,True,True,2005,"Private beach access, panoramic ocean views, m...",Welcome to paradise! This stunning villa offer...,Sandy Shores in Malibu is renowned for its exc...
6,Charlottesville,Greenwood,25 Serenity Lane,Virginia,"$850,000",7.0,4,3.5,"3,200 sqft",single-family,5 acres,True,False,True,True,1890,"Original hardwood floors, antique fireplace, r...",Step into the charm of a bygone era with this ...,Greenwood offers a peaceful escape from the ci...
7,New York City,Central Park,25 Luxe Street,NY,"$5,500,000",9.0,4,4.5,"3,500 sqft",apartment,N/A,True,False,False,True,2015,"Floor-to-ceiling windows, private terrace, mod...",Welcome to this extravagant penthouse offering...,Central Park neighborhood offers unparalleled ...
8,Boulder,Aspen Pines,22 Mountain View Road,Colorado,"$750,000",7.0,3,2.5,"2,500 sqft",single-family,1 acre,True,False,True,True,2008,"Stunning mountain views, wrap-around deck, hig...",Escape to this secluded cabin nestled in the R...,Aspen Pines is a picturesque and upscale neigh...
9,New Orleans,Garden District,24 Magnolia Avenue,Louisiana,"$2,500,000",7.0,4,3.5,"4,500 sqft",single-family,0.5 acres,True,True,True,True,1900,"Original Victorian architecture, renovated kit...",Step into a bygone era with this stunning hist...,"Nestled in the prestigious Garden District, th..."


In [70]:
#listings_df.to_csv('./data/llm_generated_real_estate_listings.csv', index=False)

In [90]:
loaded_listings_df = pd.read_csv('./data/llm_generated_real_estate_listings.csv')

## 2. Generate images for the real estate listings
### 2.1. Generate one primary image for each listing and 3 alternative images

In [139]:
client = OpenAI()

In [145]:
def generate_property_images(client: OpenAI, description: str) -> List[Tuple[str, ImagesResponse]]:
    # Define the parts of the property for which to generate images
    parts = {
        "front": "from the front of the property",
        "kitchen": "of the kitchen",
        "living_room": "of the living room, from inside the living room",
        "bedroom": "of one of the bedrooms"
    }

    # List to hold the responses for each image along with its tag
    responses = []

    # Loop through each part and generate an image
    for tag, place_description in parts.items():
        # Create the prompt using the formatted listing and the specified part
        image_template = """
        Create a realistic image of a property {place}. It should look REAL!!! Something that looks like pictures from zillow.com. Don't produce anything that looks fake or unrealistic.

        Here is a description of the property: {input}
        """
        final_prompt = image_template.format(
            place=place_description,
            input=description
        )

        # Generate the image using the client's API
        response = client.images.generate(
            model="dall-e-3",
            prompt=final_prompt,
            size="1024x1024",
            style="natural",
            quality="standard",
            n=1  # Adjust this as needed; n=1 for each call since we're looping
        )
        # Append the response along with the tag to the list
        responses.append((tag, response))

    return responses

In [146]:
def save_images(responses: List[Tuple[str, ImagesResponse]], id: int) -> None:
    """Save the images to the data folder"""
    for i, tag_response in enumerate(responses):
        image_url = tag_response[1].data[0].url
        image = requests.get(image_url).content
        with open(f"./data/images/image_{id}_{tag_response[0]}.jpg", "wb") as file:
            file.write(image)

In [147]:
#for i, row in listings_df.iterrows():
#    responses = generate_property_images(client, row['property_description'])
#    save_images(responses, i)


## 3. Embed the listings in a vector database 

### 3.1. Load the data from the CSV file

In [91]:
csv_loader = CSVLoader(
    file_path='./data/llm_generated_real_estate_listings.csv',
    #source_column='property_description',
    #metadata_columns=list(RealEstateListingType.__annotations__.keys() - {'property_description'})
)
data = csv_loader.load()
data_df = pd.read_csv('./data/llm_generated_real_estate_listings.csv')
data_df['price'] = data_df['price'].str.replace('$', '').str.replace(',', '').astype(int)

In [92]:
data[0].page_content

"city: Cambridge\nneighborhood: Harvard Square\nstreet_address: 35 Crimson Lane\nstate: MA\nprice: $5,500,000\nschool_rating: 9.0\nbedrooms: 5\nbathrooms: 4.5\nhouse_size: 6,000 sqft\nhouse_type: single-family\nlot_size: 0.75 acres\nhas_garage: True\nhas_pool: True\nhas_garden: True\nhas_view: True\nyear_built: 1998\nspecial_features: Smart home technology, home theater, sauna, rooftop terrace\nproperty_description: Welcome to the epitome of luxury living in prestigious Harvard Square. This magnificent single-family estate boasts 5 bedrooms, 4.5 bathrooms, and over 6,000 square feet of exquisitely designed living space. Enjoy breathtaking views from your rooftop terrace, relax in the sauna, or take a dip in your private pool. This home is perfect for entertaining and elegant living at its finest.\nneighborhood_description: Harvard Square is a vibrant, intellectual hub known for its historic architecture, upscale shops, and top-rated schools. With a school rating of 9, it's the ideal ne

In [93]:
data_df.head()

,city,neighborhood,street_address,state,price,school_rating,bedrooms,bathrooms,house_size,house_type,lot_size,has_garage,has_pool,has_garden,has_view,year_built,special_features,property_description,neighborhood_description
0,Cambridge,Harvard Square,35 Crimson Lane,MA,5500000,9.0,5,4.5,"6,000 sqft",single-family,0.75 acres,True,True,True,True,1998,"Smart home technology, home theater, sauna, ro...",Welcome to the epitome of luxury living in pre...,"Harvard Square is a vibrant, intellectual hub ..."
1,Exeter,Maple Woods,33 Forest View Lane,Vermont,225000,7.0,2,1.5,"1,200 sqft",single-family,0.5 acres,True,False,True,True,1978,"Cozy fireplace, Tranquil woods setting","Escape to this charming, cozy cottage nestled ...",Located in the sought-after Maple Woods neighb...
2,Boston,Roxbury,28 Main Street,MA,250000,3.0,1,1.0,650 sqft,apartment,NaN,False,False,False,False,1950,"Recently renovated, hardwood floors, city living",Welcome to this cozy and inviting 1-bedroom ap...,Roxbury is a vibrant neighborhood known for it...
3,La Jolla,Seaside Village,22 Oceanfront Drive,California,1850000,7.0,4,3.5,"3,200 sqft",single-family,"8,000 sqft",True,True,True,True,2005,"Oceanside living, stunning views, luxurious pool",Welcome to this exquisite oceanfront retreat l...,Seaside Village is known for its exclusive wat...
4,Austin,Downtown,12 Pleasant Street,Texas,650000,7.0,2,2.5,"1,500 sqft",apartment,NaN,True,False,False,True,2015,"Modern design, rooftop terrace, city views",Welcome to this ultra-modern loft apartment lo...,Downtown Austin offers a vibrant urban lifesty...


### 3.2. Generate the LLM embeddings for each listing

In [94]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

In [95]:
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

In [96]:
embeddings = embedding_model.embed_documents([listing.page_content for listing in data])

In [97]:
len(embeddings[0])

1536

### 3.3. Store the embeddings in the vector database

In [98]:
class Listings(LanceModel):
    vector: Vector(1536)
    listing_id: int
    city: str
    neighborhood: str
    street_address: str
    state: str
    price: int
    school_rating: float
    bedrooms: int
    bathrooms: float
    house_size: str
    house_type: str
    lot_size: Optional[str]
    has_garage: bool
    has_pool: bool
    has_garden: bool
    has_view: bool
    year_built: int
    special_features: str
    property_description: str


In [99]:
try:
    db = lancedb.connect("~/.lancedb")
    table_name = "real_estate_listings"

    # Clean up the existing table if necessary
    db.drop_table(table_name, ignore_missing=True)

    # Create a new table with the defined schema
    table = db.create_table(table_name, schema=Listings)
    print(f"Table '{table_name}' created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

lance_data_rows = []
for embedding, (id, row) in zip(embeddings, data_df.iterrows()):
    row_dict = row.to_dict()
    if pd.isna(row_dict['lot_size']):
        row_dict['lot_size'] = None
    # convert price to integer
    this_listing = Listings(vector=embedding, listing_id=id, **row_dict)
    lance_data_rows.append(this_listing)

# Insert the data into the table
table.add(lance_data_rows)

Table 'real_estate_listings' created successfully.


### 3.4. Query the vector database to find similar listings

In [100]:
query_text = """I'm looking for a cozy, small cottage in the woods."""
query_embedding = embedding_model.embed_documents([query_text])[0]

# Query the vector database for similar listings
query_results = table.search(query_embedding).limit(3).to_list()

# Display the results
for query_result in query_results:
    print(f"Listing ID: {query_result['listing_id']}")
    print(query_result['property_description'])
    print()

Listing ID: 1
Escape to this charming, cozy cottage nestled in the serene woods of Vermont. This inviting 2-bedroom, 1.5-bathroom home offers a peaceful retreat with a warm fireplace and a picturesque view. Enjoy the beauty of nature right at your doorstep.

Listing ID: 6
Step into the charm of a bygone era with this exquisite antique farmhouse nestled on 5 acres of picturesque land in the heart of Charlottesville, Virginia. Built-in 1890, this historic home preserves its character with original hardwood floors, an antique fireplace, and a beautifully renovated kitchen with modern appliances. Enjoy sunsets from the wrap-around porch and bask in the serenity of a truly enchanting property.

Listing ID: 8
Escape to this secluded cabin nestled in the Rocky Mountains of Colorado, offering breathtaking views of snow-capped peaks and lush forests. This charming 3-bedroom, 2.5-bathroom home features a spacious living area with a cozy fireplace, a gourmet kitchen with granite countertops, and 

### 3.5. Qeury the vector database and add a filter.

In [101]:
def get_filter_value(value: Union[str, int, float]) -> Union[str, int, float]:
    """Get the filter value in the correct format for the query"""
    if isinstance(value, str):
        return f"'{value}'"
    return value


def parse_filters(filter_dicts: List[Dict[str, Union[str, int, float]]]) -> str:
    """Parse a list of filter dictionaries into a query string"""
    filters = []
    for filter_dict in filter_dicts:
        filter_template = f"({filter_dict['field']} {filter_dict['operator']} {get_filter_value(filter_dict['value'])})"
        filters.append(filter_template)

    filter_query_str = " AND ".join(filters)
    return filter_query_str

In [102]:
query_text = """I'm looking for a cozy, small cottage in the woods of the woods."""
hard_coded_pre_filter = [
    {"field": "bedrooms", "operator": ">=", "value": 3},
    {"field": "house_type", "operator": "=", "value": "single-family"}
]
filter_query = parse_filters(hard_coded_pre_filter)
query_embedding = embedding_model.embed_documents([query_text])[0]

# Query the vector database for similar listings
query_results = table.search(query_embedding).limit(3).where(filter_query).to_list()

# Display the results
for query_result in query_results:
    print(f"Listing ID: {query_result['listing_id']}")
    print(query_result['property_description'])
    print()

Listing ID: 6
Step into the charm of a bygone era with this exquisite antique farmhouse nestled on 5 acres of picturesque land in the heart of Charlottesville, Virginia. Built-in 1890, this historic home preserves its character with original hardwood floors, an antique fireplace, and a beautifully renovated kitchen with modern appliances. Enjoy sunsets from the wrap-around porch and bask in the serenity of a truly enchanting property.

Listing ID: 8
Escape to this secluded cabin nestled in the Rocky Mountains of Colorado, offering breathtaking views of snow-capped peaks and lush forests. This charming 3-bedroom, 2.5-bathroom home features a spacious living area with a cozy fireplace, a gourmet kitchen with granite countertops, and a master suite with a private balcony. The wrap-around deck is ideal for stargazing or enjoying your morning coffee while surrounded by nature.



## 4. Create a chatbot to get a user's preferences and generate a real estate listing
### 4.1. Create a chatbot to get a user's preferences

In [103]:
#from langchain.memory import ConversationBufferMemory
#from langchain.prompts import MessagesPlaceholder
#from langchain.schema.runnable import RunnablePassthrough
#from langchain.agents.format_scratchpad import format_to_openai_functions
#from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
#from langchain.agents import AgentExecutor


In [104]:
import panel as pn  # GUI
import param

In [105]:
class RealEstateDescriptionType(TypedDict):
    property_description: str
    neighborhood_description: str


class RealEstateDescription(BaseModel):
    """Real Estate Listing Description of the property and neighborhood"""
    property_description: str = Field(
        description="A description of the property as if a real estate agent is describing it")
    neighborhood_description: str = Field(description="Description of the neighborhood that fits in one paragraph")


def create_summary_template(model: BaseModel) -> ChatPromptTemplate:
    """Generates a ChatPromptTemplate based on a Pydantic model"""

    instructions = """Summarize this real estate conversation to best describe the user's preferences in terms of property and neighborhood: {input}. You can use the additional information: {additional_info}"""

    properties = model.schema()["properties"]
    output_format = "\n".join(
        f"* {name} ({info['type']}): ({info['description']}): " for name, info in properties.items())

    return ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(instructions),
        AIMessagePromptTemplate.from_template(output_format)
    ])


In [110]:
### pn.extension()
class RealEstateAssistant(param.Parameterized):
    # User specifications
    min_bathrooms = param.Number(default=1, bounds=(1, 10))
    min_price = param.Integer(default=10000, bounds=(10000, 10000000))
    max_price = param.Integer(default=10000000, bounds=(50000, 10000000))
    min_bedrooms = param.Integer(default=1, bounds=(1, 10))

    # Chat output
    output = param.String(
        default="Welcome! I am your virtual real estate assistant. Please ask me anything about finding your new home.\n\n"
                "Assistant: What type of property interests you? (e.g., house, apartment, condo)?")
    # New Conversation Button
    new_conversation = param.Action(lambda x: x.reset_conversation(), label="New Conversation")

    def __init__(self, db_table, **params):
        super(RealEstateAssistant, self).__init__(**params)
        self.questions = [
            "Describe your ideal neighborhood or city?",
            "Do you have kids? If so, what are their ages?",
            "What should be the size of the property?",
            "What style of building do you like?",
            "Are there any specific amenities you're looking for (e.g., pool, backyard)?"
        ]
        self.question_index = 0
        self.llm_model = ChatOpenAI(temperature=0)
        summary_prompt = create_summary_template(RealEstateDescription)
        openai_real_estate_summary_function = convert_pydantic_to_openai_function(RealEstateDescription)
        llm_summary_model_with_functions = self.llm_model.bind(functions=[openai_real_estate_summary_function],
                                                               function_call={"name": "RealEstateDescription"})
        self.db_table = db_table
        self.summary_tagging_chain = summary_prompt | llm_summary_model_with_functions | JsonOutputFunctionsParser()
        self.summary = None
        self.listings = []
        self.listings_pane = pn.pane.Markdown("No listings found yet.", height=800, width=600, sizing_mode='fixed')  # To display listings

        self.layout = self.create_view()  # Store layout reference

    @param.depends('new_conversation', watch=True)
    def reset_conversation(self):
        self.min_bathrooms = 1
        self.min_price = 10000
        self.max_price = 10000000
        self.min_bedrooms = 1
        self.summary = None
        self.output = "Welcome! I am your virtual real estate assistant. Please ask me anything about finding your new home.\n\n" + \
                      "What type of property interests you? (e.g., house, apartment, condo)"

        self.question_index = 0
        self.layout = self.create_view()  # Store layout reference
        

    def user_input(self, event):
        if event.new:
            if self.question_index == len(self.questions):
                # Handling the final response after all questions

                self.output += f"\nUser: {event.new}"
                self.summary = self.generate_summary()
                self.search_listings()
                self.output += f"\n\nAssistant: Thank you for your responses. We'll get back to you with property options."
            elif self.question_index < len(self.questions):
                # Handling the normal flow of conversation
                self.output += f"\nUser: {event.new}\n\nAssistant: {self.questions[self.question_index]}"
                self.question_index += 1
            event.obj.value = ''  # Clear the input box after the response
            
    def generate_summary(self):
        try:
            other_info = {
                "min_bathrooms": self.min_bathrooms,
                "min_bedrooms": self.min_bedrooms,
                "min_price": self.min_price,
                "max_price": self.max_price
            }
            this_result = self.summary_tagging_chain.invoke({"input": self.output, "additional_info": other_info})
            self.output += f"\n\nAssistant: Thank you for your responses. Here's a summary of your needs:\nproperty description: {this_result['property_description']}\nneighborhood description: {this_result['neighborhood_description']}"
            return this_result
        except Exception as e:
            self.output += "\n\nAssistant: Failed to generate summary due to an API error."
            print(f"Error: {str(e)}")

    def create_input_box(self):
        input_box = pn.widgets.TextInput(placeholder='Type here and press enter...')
        input_box.param.watch(self.user_input, 'value')
        return input_box

    def _create_one_listing(self, listing) -> str:

        # HTML for image display
        images_html = ""
        for key in ['front', 'kitchen', 'living_room', 'bedroom']:
            images_html += f"<img src='./data/images/image_{listing['listing_id']}_{key}.jpg' alt='{key} view' style='width: 100%; max-width: 250px; padding: 5px;' />"

        return f"""
        <div style="border: 1px solid #ccc; padding: 10px; margin: 10px;">
            <h3>{listing.street_address}, {listing.city}, {listing.state}</h3>
            <div style="display: flex; flex-wrap: wrap; justify-content: space-around;">
                {images_html}
            </div>
            <p><strong>Price:</strong> ${listing['price']}</p>
            <p><strong>Bedrooms:</strong> {listing['bedrooms']} | <strong>Bathrooms:</strong> {listing['bathrooms']}</p>
            <p><strong>House Size:</strong> {listing['house_size']} | <strong>Lot Size:</strong> {listing['lot_size'] or 'N/A'}</p>
            <p><strong>Type:</strong> {listing['house_type']} | <strong>Built in:</strong> {listing['year_built']}</p>
            <p><strong>Neighborhood:</strong> {listing['neighborhood']}</p>
            <p><strong>School Rating:</strong> {listing['school_rating']} / 10</p>
            <p><strong>Features:</strong> {'Garage' if listing['has_garage'] else ''} {'Pool' if listing['has_pool'] else ''} {'Garden' if listing.has_garden else ''} {'View' if listing.has_view else ''}</p>
            <p><strong>Special Features:</strong> {listing.special_features}</p>
            <p><strong>Description:</strong> {listing.property_description}</p>
        </div>
        """
        

    def search_listings(self):
        """Search for real estate listings based on user preferences"""
        # Query the vector database for similar listings
        query_text = f"Property Description: {self.summary['property_description']}\nNeighborhood Description: {self.summary['neighborhood_description']}."
        query_embedding = embedding_model.embed_documents([query_text])[0]

        hard_coded_pre_filter = [
            {"field": "bathrooms", "operator": ">=", "value": float(self.min_bathrooms)},
            {"field": "bedrooms", "operator": ">=", "value": int(self.min_bedrooms)},
            {"field": "price", "operator": ">=", "value": int(self.min_price)},
            {"field": "price", "operator": "<=", "value": int(self.max_price)},
        ]
        filter_str = parse_filters(hard_coded_pre_filter)

        # Query the vector database for similar listings
        self.listings = self.db_table.search(query_embedding).limit(3).where(filter_str).to_list()
        

        # Display the results
        #listings_displays = [self._create_one_listing(listing) for listing in self.listings]
        #listing_display = '<br>'.join(listings_displays) if len(listings_displays) > 0 else None
        #self.listings_pane.object = listing_display if listing_display else "No listings found that match your criteria."
        
        #self.layout = self.create_view()  # Store layout reference
        


    def create_view(self):

        other_inputs = pn.Row(
            pn.Param(self.param.min_bathrooms, widgets={'min_bathrooms': {'name': 'Minimum Bathrooms'}}),
            pn.Param(self.param.min_bedrooms, widgets={'min_bedrooms': {'name': 'Minimum Bedrooms'}}),
            width=280
        )
        price_inputs = pn.Row(
            pn.Param(self.param.min_price, widgets={'min_price': {'name': 'Minimum Price'}}),
            pn.Param(self.param.max_price, widgets={'max_price': {'name': 'Maximum Price'}}),
            width=280
        )

        left_side = pn.Column(
            pn.Param(self.param, parameters=['new_conversation'],
                     widgets={'new_conversation': {'button_type': 'primary'}}),
            pn.layout.Divider(),
            other_inputs,
            price_inputs,
            pn.layout.Divider(),
            pn.pane.Markdown(self.param.output, sizing_mode='stretch_width'),
            self.create_input_box(),  # Initially add the input box
            height=800,
            width=600, 
            sizing_mode='fixed'
        )
        right_side = pn.Column(
            pn.pane.Markdown("# Real Estate Listings", margin=(0, 0, 28, 0)),
            pn.layout.Divider(),
            self.listings_pane,  # Use the listings pane here
            height=800,
            width=600,
            sizing_mode='fixed'
        )

        return pn.Row(left_side, right_side)

    def view(self):
        return self.layout

    


# Create an instance of the assistant
assistant = RealEstateAssistant(table)

# Display the application
assistant.view().servable()

Row
    [0] Column(height=800, sizing_mode='fixed', width=600)
        [0] Param(RealEstateAssistant, parameters=['new_conversation'], widgets={'new_conversation': {...})
        [1] Divider()
        [2] Row(width=280)
            [0] Param(RealEstateAssistant, parameters=['min_bathrooms'], show_name=False, widgets={'min_bathrooms': {...})
            [1] Param(RealEstateAssistant, parameters=['min_bedrooms'], show_name=False, widgets={'min_bedrooms': {...})
        [3] Row(width=280)
            [0] Param(RealEstateAssistant, parameters=['min_price'], show_name=False, widgets={'min_price': {...})
            [1] Param(RealEstateAssistant, parameters=['max_price'], show_name=False, widgets={'max_price': {...})
        [4] Divider()
        [5] Markdown(str, sizing_mode='stretch_width')
        [6] TextInput(placeholder='Type here and p...)
    [1] Column(height=800, sizing_mode='fixed', width=600)
        [0] Markdown(str, margin=(0, 0, 28, 0))
        [1] Divider()
        [2] Markdown(str, height=800, sizing_mode='fixed', width=600)

## 5. Search for real estate listings based on user preferences